## Preprocesamiento

In [2]:
import pandas as pd
from PIL import Image
import numpy as np

In [3]:
dataset = pd.read_csv('data/data.csv')

In [4]:
numeric_features = ['extent']
categorical_features = ['growth_stage', 'damage', 'season']

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

transformers_encoder_onehot = [
    ('encoder', OneHotEncoder(), categorical_features)
]
column_transformer = ColumnTransformer(transformers_encoder_onehot, remainder='passthrough')

In [6]:
dataset_encoded = column_transformer.fit_transform(dataset)
dataset = pd.DataFrame(dataset_encoded, columns=column_transformer.get_feature_names_out())

In [7]:
new_column_names = [col.replace('encoder__', '').replace('remainder__', '') for col in column_transformer.get_feature_names_out()]
dataset.columns = new_column_names

In [8]:
dataset = dataset.drop(columns=['ID'])

In [9]:
dataset.head()

,growth_stage_F,growth_stage_M,growth_stage_S,growth_stage_V,damage_DR,damage_DS,damage_FD,damage_G,damage_ND,damage_PS,damage_WD,damage_WN,season_LR2020,season_LR2021,season_SR2020,season_SR2021,filename,extent
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,L427F01330C01S03961Rp02052.jpg,0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,L1083F00930C39S12674Ip.jpg,0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24_initial_1_1463_1463.JPG,0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,L341F00167C01S00324Rp14178.jpg,60
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,L1084F02394C39S13931Ip.jpg,0


In [10]:
def resizeImg(path, size):
    img = Image.open(path)

    width, height = img.size
    aspectRatio = width / height

    if aspectRatio < 1:
        nHeight = size
        nWidth = int(size * aspectRatio)
    else:
        nWidth = size
        nHeight = int(size / aspectRatio)

    resizedImg = img.resize((nWidth, nHeight), Image.LANCZOS)

    newImg = Image.new("RGB", (size, size), (255, 255, 255))

    xOffset = (size - nWidth) // 2
    yOffset = (size - nHeight) // 2
    newImg.paste(resizedImg, (xOffset, yOffset))

    return newImg

In [11]:
import os
from tqdm import tqdm

# Ruta donde se encuentran las imágenes
image_path = "data/train/"
image_size = 128
output_file = "resized_images.npy"

def load_and_resize_images(df, path, size):
    images = []
    for filename in tqdm(df['filename']):
        img = resizeImg(os.path.join(path, filename), size)
        img_array = np.array(img) / 255.0
        images.append(img_array)
    return np.array(images)

# Verificar si ya existe el archivo .npy
if not os.path.exists(output_file):
    images = load_and_resize_images(dataset, image_path, image_size)
    np.save(output_file, images)  # Guardar el archivo .npy
else:
    images = np.load(output_file)  # Cargar el archivo .npy


100%|██████████| 26068/26068 [16:31<00:00, 26.29it/s] 


## CNN

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Cargar el modelo preentrenado VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
# Extraer las características de la última capa convolucional
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

# Extraer características
features = model.predict(images)
# Aplanar las características para usarlas en Random Forest
features_flat = features.reshape(len(features), -1)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
815/815 ━━━━━━━━━━━━━━━━━━━━ 1581s 2s/step


In [14]:
# Convertir las características a un DataFrame y unir con el dataset original
features_df = pd.DataFrame(features_flat)
df_features = pd.concat([dataset.reset_index(drop=True), features_df], axis=1)
df_features.drop(columns=["filename"], inplace=True)  # Retirar la columna 'filename' ya que ahora tenemos las features

In [26]:
# Guardar el DataFrame completo con las características y etiquetas
df_features.to_csv("final_dataset.csv", index=False)

## Random Forest

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Dividir datos
X = df_features.drop(columns=["extent"])
y = df_features["extent"]

# Convertir nombres de columnas a tipo string
X.columns = X.columns.astype(str)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model_rf.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")

RMSE: 8.726778820651981


c:\Users\Pablo Zamora\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluar el modelo
y_pred = model_rf.predict(X_test)

# Calcular RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")

# Calcular R²
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

RMSE: 8.726778820651981
R²: 0.7901191755276311


c:\Users\Pablo Zamora\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [25]:
# Visualizando el primer registro del conjunto de prueba
single_example = X_test.iloc[0].values.reshape(1, -1)  # Redimensionar para que sea un array de 2D

# Hacer la predicción
single_prediction = model_rf.predict(single_example)

print(f"\nValor real para el ejemplo: {y_test.iloc[0]}")
print(f"Predicción para el ejemplo: {single_prediction[0]}")


Valor real para el ejemplo: 30
Predicción para el ejemplo: 34.0


c:\Users\Pablo Zamora\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [27]:
import joblib

# Guardar el modelo entrenado
joblib.dump(model_rf, "CNN_RandomForest/random_forest_model.pkl")

['CNN_RandomForest/random_forest_model.pkl']